In [ ]:
import os
import sys

sys.path.append('../../')  # Add the path to the my_packages module
os.environ['EXPERIMENT_DB_NAME'] = "few_shot_experiments"
from my_packages.db_service.error_service import delete_error_docs, errors_to_df, pretty_print_errors
from my_packages.evaluation.midio_compiler import compile_code
from my_packages.db_service.experiment_service import experiment_exists, pretty_print_experiment_collections, run_experiment_quality_checks, setup_experiment_collection


experiment_name = f"regular_coverage_5_shot"
syntax_filter = {
    "experiment_name": experiment_name,
    "model": "Phi4:14b",
    "error_type": "syntax",
}
df = errors_to_df(experiment_name, filter=filter)
# extract "CompilerError - .." from list of strings in errors_msg field


print(df)



         model_name task_id  candidate_id  metric error_type  \
0     phi4:14b-fp16       8             5  syntax     syntax   
1     phi4:14b-fp16       8             8  syntax     syntax   
2     phi4:14b-fp16      10             3  syntax     syntax   
3     phi4:14b-fp16      10             6  syntax     syntax   
4     phi4:14b-fp16      10             8  syntax     syntax   
...             ...     ...           ...     ...        ...   
4223   qwq:32b-fp16      16             1   tests     syntax   
4224   qwq:32b-fp16      18             1   tests   semantic   
4225   qwq:32b-fp16      34             1   tests     syntax   
4226   qwq:32b-fp16      42             1   tests   semantic   
4227   qwq:32b-fp16      48             1   tests     syntax   

                                              error_msg  \
0     [ERROR compiler::frontend::code_gen: 3182: Err...   
1     [ERROR compiler::frontend::parser: 1372: Error...   
2     [ERROR compiler::frontend::parser: 1372: Error..

In [2]:

sys.path.append('../../')  # Add the path to the my_packages module
from my_packages.db_service.data_visualization import visualize_error_flow_for_model



# Assuming the experiment 'GPT4_signature_exp1' exists,
# and you evaluated 100 candidates for model 'GPT-4':
# visualize_error_flow_for_model(experiment_name, "gpt-4o")
pretty_print_experiment_collections(experiment_name, filter={"eval_method": "hold_out"})


NameError: name 'sys' is not defined